# Qiskit Aer: Pulse Simulator

Notes:
- Currently have a section on creating transmon models using the system generators
- Have imported examples from Dave's notebook with one of these generated models, and am trying to write some stuff around them
    - Question: it would simplify things to remove the measurement pulses - they have no influence on the simulation, and would declutter a bit of it. Should we keep or get rid of them?
- Doing a simulation follow-up to the previous point: set up a system, create some sort of basic `Schedule`, show how to run it. Not sure if this should be something that generates an interesting plot, or just something that shows how to run the simulator.
- Example of running something on a device, then running it on the simulator, where the `PulseSystemModel` has been generated from the backend
    - This needs to be a calibration procedure, preferably similar to Dave's notebook for ease

## Introduction

This notebook shows how to use the Aer pulse simulator, which simulates an OpenPulse `Schedule` on a quantum system at the Hamiltonian level.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from qiskit import IBMQ
import qiskit

# for constructing and specifying pulse experiments
import qiskit.pulse as pulse
import qiskit.pulse.pulse_lib as pulse_lib
from qiskit.compiler import assemble
from qiskit.qobj.utils import MeasLevel, MeasReturnType

# system model object
from qiskit.providers.aer.openpulse.pulse_system_model import PulseSystemModel

# function for construct transmon device models
from qiskit.providers.aer.openpulse.transmon_model_generators import transmon_system_model

## Models of pulse devices

The physical model is stored in a `PulseSystemModel` object. This object stores all information required to specify a physical system ready for pulse control.

### Generating transmon models

The function `transmon_system_model` constructs a `PulseSystemModel` for a transmon system specified in terms of individual qubit terms, and a coupling map with coupling strenghts.

Each single transmon is specified by a frequency $\nu$, anharmonicity $\alpha$, and drive strength $r$, which results in the Hamiltonian terms:
\begin{equation}
    \pi(2\nu - \alpha)a^\dagger a + \pi \alpha (a^\dagger a)^2 + 2 \pi r (a + a^\dagger) \times D(t),
\end{equation}
where $D(t)$ is the drive signal for the qubit.

Specifying a coupling for transmon pair $(l,k)$ with coupling strength $j$, results in the exchange coupling Hamiltonian term:
\begin{equation}
    2 \pi j (a_l \otimes a_k^\dagger + a_l^\dagger \otimes a_k).
\end{equation}

Finally, control channels are generated for doing cross-resonance drives between each pair of coupled transmons.

In [2]:
# number of qubits and cutoff dimensions
num_transmons = 2
dim_transmons = 3

# frequencies for transmon drift terms, harmonic term and anharmonic term
transmon_freqs = [5.0e9, 5.1e9]
anharmonicity_freqs = [-0.33e9, -0.33e9]

# transmon drive strengths
drive_strengths = [0.02e9, 0.02e9]

# specify coupling as a dictionary (it says qubits 0 and 1 are coupled with a coefficient 0.02)
coupling_dict = {(0,1): 0.01e9}

# time 
dt = 1e-9

# create the model
two_transmon_model, cr_idx_dict = transmon_system_model(num_transmons=num_transmons, 
                                                        dim_transmons=dim_transmons,
                                                        transmon_freqs=transmon_freqs,
                                                        anharm_freqs=anharmonicity_freqs,
                                                        drive_strengths=drive_strengths,
                                                        coupling_dict=coupling_dict,
                                                        dt=dt)

The returned items are: 
- `system_model`, an instance of `PulseSystemModel` consumable by the simulator representing the specified transmon system model. 
- `cr_idx_dict`, a dict storing information on the index of the channel for a given CR drive. Specifically, when two qubits are coupled in the model, two CR drive channels are created for doing CR drives in both directions, and `cr_idx_dict` keeps track of the indices for these channels. E.g. in the above we specified the coupling `(0,1)`. When performing a CR drive on qubit 0 with target 1, use u channel with index `cr_idx_dict[(0,1)]`, and when performing a CR drive on qubit 1 with target 0, use u channel with index `cr_idx_dict[(1,0)]`

## Example 1: Rabi experiment on qubit $0$

To run an experiment on the simulator, construct pulse `Schedule` objects as for a real device. Here, we do a Rabi experiment, similar to the `ignis/1a_calibrating_a_qubit.ipynb` tutorial.

In [3]:
# qubit to use for exeperiment
qubit = 0
# exp configuration
exps = 41
shots = 512

# Rabi pulse
drive_amps = np.linspace(0, 0.9, exps)
drive_samples = 128
drive_sigma = 16

# Measurement pulse
meas_amp = 0.025
meas_samples = 1200
meas_sigma = 4
meas_risefall = 25

# Measurement pulse (common for all experiment)
meas_pulse = pulse_lib.gaussian_square(duration=meas_samples, amp=meas_amp,
                                       sigma=meas_sigma, risefall=meas_risefall, 
                                       name='meas_pulse')
acq_cmd=pulse.Acquire(duration=meas_samples)

acquire_channels = [pulse.AcquireChannel(0), pulse.AcquireChannel(1)]
memoryslots = [pulse.MemorySlot(0), pulse.MemorySlot(1)]

# create measurement schedule
measure_and_acquire = meas_pulse(pulse.MeasureChannel(0)) | acq_cmd(acquire_channels, memoryslots)

# Create schedule
schedules = []
for ii, drive_amp in enumerate(drive_amps):
    # drive pulse
    rabi_pulse = pulse_lib.gaussian(duration=drive_samples, 
                                    amp=drive_amp, 
                                    sigma=drive_sigma, name='rabi_pulse_%d' % ii)
    
    # add commands to schedule
    schedule = pulse.Schedule(name='rabi_exp_amp_%s' % drive_amp)
    
    schedule += rabi_pulse(pulse.DriveChannel(qubit))
    #schedule += measure_and_acquire << schedule.duration
 
    schedules.append(schedule)

### Assemble the PulseQobj and run the simulation

Assemble the schedules. When assembling for the simulator, pass the `PulseSimulator` as the backend.

In [4]:
backend_sim = qiskit.Aer.get_backend('pulse_simulator')

rabi_qobj = assemble(schedules,
                     backend=backend_sim,
                     meas_level=1, 
                     meas_return='avg',
                     shots=shots)

/Users/dpuzzuoli/anaconda3/envs/QiskitDev/lib/python3.7/site-packages/qiskit/providers/models/pulsedefaults.py:155: UserWarning: `qubit_freq_est` and `meas_freq_est` now have units of Hertz(Hz) rather than gigahertz(GHz).
  warnings.warn('`qubit_freq_est` and `meas_freq_est` now have units of '


Run the simulation

In [5]:
sim_result = backend_sim.run(rabi_qobj, two_transmon_model).result()

/Users/dpuzzuoli/anaconda3/envs/QiskitDev/lib/python3.7/site-packages/qiskit/providers/aer/openpulse/qobj/digest.py:94: UserWarning: Warning: qubit_lo_freq was not specified in PulseQobj or in PulseSystemModel,              so it is beign automatically determined from the drift Hamiltonian.
  so it is beign automatically determined from the drift Hamiltonian.')


UnboundLocalError: local variable 'psi' referenced before assignment

Note, the above warning is stating that the `qubit_lo_freq` is being computed from the Hamiltonian of the system, as the user did not specify one.

### Plot the results

In [ ]:
# Extract qubit populations

amp_data_Q0 = []
amp_data_Q1 = []

for exp_idx in range(len(drive_amps)):
    exp_mem = sim_result.get_memory(exp_idx)
    amp_data_Q0.append(np.abs(exp_mem[0]))
    amp_data_Q1.append(np.abs(exp_mem[1]))
    

#Fit the data
fit_func = lambda x,A,B,T,phi: (A*np.cos(2*np.pi*x/T+phi)+B)
fitparams, conv = curve_fit(fit_func, drive_amps, amp_data_Q0, [0.5,0.5,0.6,1.5])

#get the pi amplitude
pi_amp = (fitparams[3])*fitparams[2]/2/np.pi

plt.plot(drive_amps, amp_data_Q0, label='Q0')
plt.plot(drive_amps, amp_data_Q1, label='Q1')
plt.plot(drive_amps, fit_func(drive_amps, *fitparams), color='black', linestyle='dashed', label='Fit')
plt.axvline(pi_amp, color='black', linestyle='dashed')
plt.legend()
plt.xlabel('Pulse amplitude, a.u.', fontsize=20)
plt.ylabel('Signal, a.u.', fontsize=20)
plt.title('Rabi on Q0', fontsize=20)
plt.grid(True)

print('Pi Amplitude %f'%(pi_amp))

# Example 2: Cross resonance drive

Simulate a cross-resonance drive on qubit $1$, with target qubit $0$. Note, this requires the Rabi pulse to be calibrated.

### Set up the experiments

Note, to do a CR drive on qubit $1$ with target qubit $0$, we need the the index of the control channel. This can be retrieved from the `cr_idx_dict` originally returned when the transmon model was generated. The index of the channel is the value corresponding to key `(1,0)`. (In general, the index of the control channel for a CR drive on qubit `l` with target `k` is `cr_idx_dict[(l,k)]`.)

In [ ]:
cr_10_idx = cr_idx_dict[(1,0)]

Set up the pulses.

In [ ]:
# exp configuration
exps = 41
shots = 512

# Rabi pulse
cr_drive_amps = np.linspace(0, 0.9, exps)
cr_drive_samples = 128*3
cr_drive_sigma = 4


# Create schedule
schedules = []
for ii, cr_drive_amp in enumerate(cr_drive_amps):
    # drive pulse
    cr_rabi_pulse = pulse_lib.gaussian_square(duration=cr_drive_samples, 
                                    amp=cr_drive_amp, 
                                    risefall=cr_drive_sigma*4,
                                    sigma=cr_drive_sigma, name='rabi_pulse_%d' % ii)
    
    # add commands to schedule
    schedule = pulse.Schedule(name='cr_rabi_exp_amp_%s' % cr_drive_amp)
    
    # do cr drive on the correct
    schedule += cr_rabi_pulse(pulse.ControlChannel(cr_10_idx)) 
    schedule += measure_and_acquire << schedule.duration
 
    schedules.append(schedule)

As before, set up the `PulseQobj`, run the simulation, and plot the results.

In [ ]:
cr_rabi_qobj = assemble(schedules,
                        backend=backend_sim,
                        meas_level=1, 
                        meas_return='avg',
                        shots=shots)

In [ ]:
sim_result = backend_sim.run(cr_rabi_qobj, two_transmon_model).result()

# Generating transmon models from backends

Alternatively, one may wish to model from an IBM backend. In this case, it is possible to construct one from the backend. The `ibmq_armonk` backend is a single qubit, pulse-enabled device.

In [ ]:
provider = IBMQ.load_account()
provider.backends()
armonk_backend = provider.get_backend('ibmq_armonk')

We can construct a model directly from the backend using the `.from_backend` constructor.

In [ ]:
armonk_model = PulseSystemModel.from_backend(armonk_backend)

armonk_model.hamiltonian._variables

Note: the above is problematic, as the numbers for the hamiltonian in the backend don't necessarily even have values, e.g. the drive strength in the armonk backend is set to $0$, presumably as there is no well known 'reasonable' number to put in. Furthermore, the units of the qubit frequency are in $rad \times$ GHz still, so are inconsistent with the Hz convention.

The easiest way around this might be to re-introduce the 'set variable' method back into `HamiltonianModel`.

# Running a pulse schedule on the simulator

First, construct a list of pulse experiments in `Schedule` objects. 

Note: This is taken directly from
https://github.com/Qiskit/qiskit-iqx-tutorials/blob/master/qiskit/advanced/ignis/1a_calibrating_a_qubit.ipynb
One option is to not run it on the device here at all, but to just reference that tutorial. On the other hand, if that tutorial changes it could mess things up here.

In [ ]:
defaults = armonk_backend.defaults()
config = armonk_backend.configuration()

circ_inst_map = defaults.circuit_instruction_map

measure = circ_inst_map.get('measure', qubits=config.meas_map[0])

# qubit to use for experiment
qubit = 0

# exp configuration
exps = 64
shots = 512

# Rabi pulse
drive_amps = np.linspace(0, 1.0, exps)
drive_samples = 2048
drive_sigma = 256

# scaling factor for data returned by backend
# note: You may have to adjust this for the backend you use
scale_factor=1e-10

# Create schedule
schedules = []
for ii, drive_amp in enumerate(drive_amps):
    # drive pulse
    rabi_pulse = pulse_lib.gaussian(duration=drive_samples, amp=drive_amp, sigma=drive_sigma, name='rabi_pulse_%d' % ii)
    
    # add commands to schedule
    schedule = pulse.Schedule(name='Rabi Experiment at drive amp = %s' % drive_amp)
    
    schedule |= rabi_pulse(pulse.DriveChannel(qubit))
    schedule |= measure << schedule.duration
 
    schedules.append(schedule)
    
rabi_qobj = assemble(schedules, armonk_backend, 
                     meas_level=MeasLevel.KERNELED, 
                     meas_return=MeasReturnType.AVERAGE, 
                     shots=shots)

Run the job

In [ ]:
job = armonk_backend.run(rabi_qobj)

In [ ]:
job.status()

In [ ]:
rabi_result = job.result(timeout=3600)

In [ ]:
qubit_rabi_data = np.ones(exps, dtype=np.complex_)
for i in range(exps):
    qubit_rabi_data[i] = rabi_result.get_memory(i)[qubit]*scale_factor
    
    
def get_amplitude(vec):
    i_signal = np.imag(vec)
    r_signal = np.real(vec)

    mvec = [np.mean(r_signal), np.mean(i_signal)]

    src_mat = np.vstack((r_signal - mvec[0], i_signal - mvec[1])).T
    (_, _, v_mat) = np.linalg.svd(src_mat)

    dvec = v_mat[0, 0:2]

    if dvec.dot(mvec) < 0:
        dvec = -dvec

    return src_mat.dot(dvec)    

rabi_amp_data = get_amplitude(qubit_rabi_data)

fit_func = lambda x,A,B,Omega,phi: (A*np.cos(2*np.pi*x/Omega+phi)+B)

#Fit the data
fitparams, conv = curve_fit(fit_func, drive_amps, rabi_amp_data, [7.0,0.0,1.0,0.3])

#get the pi amplitude
pi_amp = (np.pi-fitparams[3])*fitparams[2]/4/np.pi
#pi_amp = (fitparams[3])*fitparams[2]/2/np.pi

plt.scatter(drive_amps, rabi_amp_data)
plt.plot(drive_amps, fit_func(drive_amps, *fitparams), color='red')
plt.axvline(pi_amp, color='black', linestyle='dashed')
plt.xlim(0, 1)
plt.ylim(-8, 8)
plt.xlabel('Pulse amplitude, a.u.', fontsize=20)
plt.ylabel('Signal, a.u.', fontsize=20)
plt.title('Rough Pi Amplitude Calibration', fontsize=20)

print('Pi Amplitude %f'%(pi_amp))

## Run it on the simulator

In [ ]:
pi_amp = 0.305795

In [ ]:
# number of qubits and cutoff dimensions
num_transmons = 1
dim_transmons = 2

# frequencies for transmon drift terms, harmonic term and anharmonic term
transmon_freqs = getattr(defaults, 'qubit_freq_est')
anharmonicity_freqs = [0] # as we are simulating on 2 levels, has no effect

# estimate drive strength
#drive_strengths = [0.002]
drive_strengths = [np.real(pi_amp*sum(rabi_pulse.samples)*getattr(config,'dt')/(2*np.pi))]

# specify coupling as a dictionary (it says qubits 0 and 1 are coupled with a coefficient 0.02)
coupling_dict = {}

# time 
dt = getattr(config, 'dt')*1e-9
#dt = 1.
# create the model
armonk_model, cr_idx_dict = transmon_system_model(num_transmons=num_transmons, 
                                                  dim_transmons=dim_transmons,
                                                  transmon_freqs=transmon_freqs,
                                                  anharm_freqs=anharmonicity_freqs,
                                                  drive_strengths=drive_strengths,
                                                  coupling_dict=coupling_dict,
                                                  dt=dt)

In [ ]:
# Get pulse simulator backend
backend_sim = qiskit.Aer.get_backend('pulse_simulator')

In [ ]:
rabi_qobj_sim = assemble(schedules, 
                         backend=backend_sim, 
                         meas_level=MeasLevel.KERNELED, 
                         meas_return=MeasReturnType.AVERAGE, 
                         shots=shots)

In [ ]:
sim_result = backend_sim.run(rabi_qobj_sim, armonk_model).result()

In [ ]:
qubit_rabi_data = np.ones(exps, dtype=np.complex_)
for i in range(exps):
    qubit_rabi_data[i] = sim_result.get_memory(i)[qubit]*10#*scale_factor
    
    
def get_amplitude(vec):
    i_signal = np.imag(vec)
    r_signal = np.real(vec)

    mvec = [np.mean(r_signal), np.mean(i_signal)]

    src_mat = np.vstack((r_signal - mvec[0], i_signal - mvec[1])).T
    (_, _, v_mat) = np.linalg.svd(src_mat)

    dvec = v_mat[0, 0:2]

    if dvec.dot(mvec) < 0:
        dvec = -dvec

    return src_mat.dot(dvec)    

rabi_amp_data = get_amplitude(qubit_rabi_data)

fit_func = lambda x,A,B,Omega,phi: (A*np.cos(2*np.pi*x/Omega+phi)+B)

#Fit the data
fitparams, conv = curve_fit(fit_func, drive_amps, rabi_amp_data, [7.0,0.0,1.0,0.3])

#get the pi amplitude
pi_amp = (np.pi-fitparams[3])*fitparams[2]/4/np.pi
#pi_amp = (fitparams[3])*fitparams[2]/2/np.pi

plt.scatter(drive_amps, rabi_amp_data)
plt.plot(drive_amps, fit_func(drive_amps, *fitparams), color='red')
plt.axvline(pi_amp, color='black', linestyle='dashed')
plt.xlim(0, 1)
plt.ylim(-8, 8)
plt.xlabel('Pulse amplitude, a.u.', fontsize=20)
plt.ylabel('Signal, a.u.', fontsize=20)
plt.title('Rough Pi Amplitude Calibration', fontsize=20)

print('Pi Amplitude %f'%(pi_amp))

In [ ]:
armonk_model.hamiltonian._system

In [ ]:
len([[]])

In [ ]:
sum(rabi_pulse.samples)*getattr(config,'dt')/(2*np.pi)

In [ ]:
getattr(config,'dt')

In [ ]:
dt

In [ ]:
drive_strengths